In [1]:
import sys
import polars as pl
import boto3
import time
from IPython.display import display
import datetime
import math
from pathlib import PurePosixPath
print(f"{sys.version=}")

sys.version='3.14.0 (main, Oct  7 2025, 09:34:52) [Clang 17.0.0 (clang-1700.3.19.1)]'


In [2]:
s3 = boto3.resource(
    's3',
    endpoint_url='http://localhost:9000',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False
)
storage_options = {
    "aws_endpoint_url": "http://localhost:9000", # Важно: aws_endpoint_url (иногда endpoint_url)
    "aws_access_key_id": "minioadmin",
    "aws_secret_access_key": "minioadmin",
    "aws_region": "us-east-1", # Для MinIO часто можно оставить us-east-1
    "aws_allow_http": "true",  # Разрешить HTTP (без SSL)
}

# Список silver S3

In [3]:
def silver_path2dt(obj):
    return datetime.datetime(**{item.split("=",1)[0]: int(item.split("=",1)[1]) for item in PurePosixPath(obj.key).parts if "=" in item})
bucket = s3.Bucket('silver')
objects = sorted(bucket.objects.all(), key=silver_path2dt)

In [4]:
objects[0], objects[1]

(s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=13/part-26e26c26-3168-436a-833a-56c4e58a7739.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=14/part-924f2112-50df-4b05-8cb5-b356c06fa286.parquet'))

In [5]:
total_size_mb = 0
for obj in objects:
    key = obj.key
    size_mb = obj.size//1e+6
    total_size_mb += size_mb
    print(key, size_mb, "mb,", "last_modified", obj.last_modified)

hh/vacancies/year=2025/month=11/day=13/part-26e26c26-3168-436a-833a-56c4e58a7739.parquet 0.0 mb, last_modified 2025-12-13 16:40:19.312000+00:00
hh/vacancies/year=2025/month=11/day=14/part-924f2112-50df-4b05-8cb5-b356c06fa286.parquet 4.0 mb, last_modified 2025-12-13 16:40:49.097000+00:00
hh/vacancies/year=2025/month=11/day=15/part-18d47697-c0b6-4fca-b919-161d3d896ec2.parquet 1.0 mb, last_modified 2025-12-13 16:40:56.821000+00:00
hh/vacancies/year=2025/month=11/day=16/part-a7b73087-b7f8-471c-a9db-2836d35e38e1.parquet 0.0 mb, last_modified 2025-12-13 16:41:03.749000+00:00
hh/vacancies/year=2025/month=11/day=17/part-b1ed24de-492f-4ee4-bcf0-f3522446bc48.parquet 7.0 mb, last_modified 2025-12-13 16:41:56.598000+00:00
hh/vacancies/year=2025/month=11/day=18/part-c4830da7-58f7-423a-a125-fc9e28311284.parquet 6.0 mb, last_modified 2025-12-13 16:42:35.990000+00:00
hh/vacancies/year=2025/month=11/day=19/part-b61f7d56-842a-4f58-807d-86e3bef02e2e.parquet 6.0 mb, last_modified 2025-12-13 16:43:16.05300

In [6]:
print('Silver layer size', total_size_mb, 'mb')

Silver layer size 239.0 mb


# Общее кол-во вакансий в S3

In [7]:
for obj in objects:
    schema = pl.scan_parquet(f"s3://silver/{obj.key}",storage_options=storage_options).collect_schema()
    print(obj, len(schema))

s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=13/part-26e26c26-3168-436a-833a-56c4e58a7739.parquet') 51
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=14/part-924f2112-50df-4b05-8cb5-b356c06fa286.parquet') 51
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=15/part-18d47697-c0b6-4fca-b919-161d3d896ec2.parquet') 51
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=16/part-a7b73087-b7f8-471c-a9db-2836d35e38e1.parquet') 51
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=17/part-b1ed24de-492f-4ee4-bcf0-f3522446bc48.parquet') 51
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=18/part-c4830da7-58f7-423a-a125-fc9e28311284.parquet') 51
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=19/part-b61f7d56-842a-4f58-807d-86e3bef02e2e.parquet') 51
s3.ObjectSummary(bucket_name='silv

In [8]:
df_lazy = pl.scan_parquet(
    [f"s3://silver/{obj.key}" for obj in objects],
    storage_options=storage_options,
)

In [9]:
df_lazy.select(
    pl.col('id').count().alias('id_cnt'),
    pl.col('id').unique().count().alias('unique_id_cnt')
).collect()

id_cnt,unique_id_cnt
u32,u32
1150357,976245


In [10]:
df_lazy.tail(1).collect().glimpse()

Rows: 1
Columns: 51
$ id                                    <str> '128744625'
$ name                                  <str> 'Продавец-кассир (ул. Бетховена)'
$ schedule                        <struct[2]> {'id': 'shift', 'name': 'Сменный график'}
$ working_time_modes        <list[struct[2]]> []
$ working_time_intervals    <list[struct[2]]> []
$ working_days              <list[struct[2]]> []
$ working_hours             <list[struct[2]]> [{'id': 'HOURS_10', 'name': '10\xa0часов'}]
$ work_schedule_by_days     <list[struct[2]]> [{'id': 'FOUR_ON_TWO_OFF', 'name': '4/2'}]
$ fly_in_fly_out_duration   <list[struct[2]]> []
$ is_adv_vacancy                       <bool> False
$ internship                           <bool> False
$ accept_temporary                     <bool> False
$ accept_incomplete_resumes            <bool> True
$ premium                              <bool> False
$ has_test                             <bool> False
$ show_contacts                        <bool> True
$ response_letter

# Сверка схем Bronze & Silver

In [11]:
check_report_dt = datetime.datetime(year=2025, month=12, day=10)

In [12]:
bronze_lazy = pl.scan_ndjson(f"s3://bronze/hh/vacancies/date={check_report_dt.strftime('%Y-%m-%d')}/*.jsonl.gz", storage_options=storage_options)
bronze_cols = set(bronze_lazy.collect_schema().names())

In [44]:
report_dt_obj = next(iter(s3.Bucket('silver').objects.filter(Prefix="hh/vacancies/"+check_report_dt.strftime("year=%Y/month=%m/day=%d")).all()))
silver_s3_path = report_dt_obj.key
silver_lazy = pl.scan_parquet(f"s3://silver/{silver_s3_path}", storage_options=storage_options)
silver_cols = set(silver_lazy.collect_schema().names())

In [45]:
silver_cols - bronze_cols

{'brand_snippet',
 'branding',
 'created_at_offset',
 'created_at_utc',
 'published_at_offset',
 'published_at_utc',
 'video_vacancy'}

In [46]:
"brand_snippet" in bronze_cols

False

In [48]:
for report_dt in {"2025-12-08", "2025-12-09", "2025-12-10", "2025-12-11", "2025-12-12", "2025-12-13", "2025-12-14", "2025-12-15"}:
    print(report_dt)
    for bronze_obj in s3.Bucket('bronze').objects.filter(Prefix=f"hh/vacancies/date={report_dt}"):
        _bronze_lazy = pl.scan_ndjson(f"s3://bronze/{bronze_obj.key}/*.jsonl.gz", storage_options=storage_options)
        print("\t", "brand_snippet" in set(bronze_lazy.collect_schema().names()))

2025-12-11
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
2025-12-09
	 False
	 False
	 False
	 False
	 False
	 False
	 False
2025-12-15
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
2025-12-13
	 False
	 False
	 False
	 False
	 False
	 False
2025-12-14
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
2025-12-08
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
2025-12-12
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
	 False
202

# Lil EDA

In [49]:
def silver_path2dt(obj):
    return datetime.datetime(**{item.split("=",1)[0]: int(item.split("=",1)[1]) for item in PurePosixPath(obj.key).parts if "=" in item})
bucket = s3.Bucket('silver')
objects = sorted(bucket.objects.all(), key=silver_path2dt)
objects

[s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=13/part-26e26c26-3168-436a-833a-56c4e58a7739.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=14/part-924f2112-50df-4b05-8cb5-b356c06fa286.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=15/part-18d47697-c0b6-4fca-b919-161d3d896ec2.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=16/part-a7b73087-b7f8-471c-a9db-2836d35e38e1.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=17/part-b1ed24de-492f-4ee4-bcf0-f3522446bc48.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=18/part-c4830da7-58f7-423a-a125-fc9e28311284.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=19/part-b61f7d56-842a-4f58-807d-86e3bef02e2e.parquet'),
 s3.ObjectSummary(bucket_name='silver', k

In [52]:
df_lazy = (
    pl.scan_parquet([f"s3://silver/{obj.key}" for obj in objects], storage_options=storage_options)

    # Published Date transfromation from UTC to Local
    .with_columns(
        published_at_local = (
                pl.col('published_at_utc').dt.replace_time_zone(None) +
                pl.duration(minutes=pl.col('published_at_offset')
            )
        )
    )
    .with_columns(
        published_date=pl.col('published_at_local').dt.date()
    )
)

In [65]:
df_lazy.select("published_at_utc", "published_at_offset", "published_at_local").head(100).collect()

published_at_utc,published_at_offset,published_at_local
datetime[μs],i16,datetime[μs]
2025-11-13 16:49:01,0,2025-11-13 16:49:01
2025-11-13 20:17:21,0,2025-11-13 20:17:21
2025-11-13 20:18:25,0,2025-11-13 20:18:25
2025-11-13 20:20:11,0,2025-11-13 20:20:11
2025-11-13 20:08:46,0,2025-11-13 20:08:46
…,…,…
2025-11-13 20:55:24,0,2025-11-13 20:55:24
2025-11-13 18:15:54,0,2025-11-13 18:15:54
2025-11-13 17:59:36,0,2025-11-13 17:59:36


In [58]:
df_lazy.select(
    pl.col('id').count(),
    pl.col('id').unique().count().alias('unique_id')
).collect()

id,unique_id
u32,u32
1150357,976245


In [64]:
dups = df_lazy.filter(pl.col("id").is_duplicated()).sort(by='id')
dups.head(20).select("id", "name", 

id,name,schedule,working_time_modes,working_time_intervals,working_days,working_hours,work_schedule_by_days,fly_in_fly_out_duration,is_adv_vacancy,internship,accept_temporary,accept_incomplete_resumes,premium,has_test,show_contacts,response_letter_required,show_logo_in_search,archived,night_shifts,url,alternate_url,apply_alternate_url,response_url,adv_response_url,immediate_redirect_url,salary,salary_range,employer,department,type,employment,employment_form,experience,area,address,work_format,relations,professional_roles,contacts,adv_context,sort_point_distance,snippet,branding,insider_interview,video_vacancy,brand_snippet,published_at_utc,created_at_utc,published_at_offset,created_at_offset,published_at_local,published_date
str,str,struct[2],list[struct[2]],list[struct[2]],list[struct[2]],list[struct[2]],list[struct[2]],list[struct[2]],bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,str,str,str,str,str,str,struct[4],struct[6],struct[9],struct[2],struct[2],struct[2],struct[2],struct[2],struct[3],struct[10],list[struct[2]],list[str],list[struct[2]],struct[3],str,f64,struct[2],struct[2],struct[2],struct[7],struct[7],datetime[μs],datetime[μs],i16,i16,datetime[μs],date
"""100055445""","""Слесарь механосборочных работ …","{""shift"",""Сменный график""}",[],[],[],"[{""HOURS_12"",""12 часов""}]","[{""THREE_ON_THREE_OFF"",""3/3""}]",[],false,false,false,true,false,false,true,false,true,false,true,"""https://api.hh.ru/vacancies/10…","""https://hh.ru/vacancy/10005544…","""https://hh.ru/applicant/vacanc…",null,null,null,"{null,135000,""RUR"",true}","{null,135000,""RUR"",true,{""MONTH"",""За месяц""},{""TWICE_PER_MONTH"",""Два раза в месяц""}}","{""611846"",""«Научно-производственное объединение «Северо-Западный региональный центр Концерна ВКО «Алмаз-Антей» - Обуховский завод»"",""https://api.hh.ru/employers/611846"",""https://hh.ru/employer/611846"",{""https://img.hhcdn.ru/employer-logo-original-round/6085704.png"",""https://img.hhcdn.ru/employer-logo-round/6085705.png"",""https://img.hhcdn.ru/employer-logo-round/6085706.png""},""https://api.hh.ru/vacancies?employer_id=611846"",1,false,true}",null,"{""open"",""Открытая""}","{""full"",""Полная занятость""}","{""FULL"",""Полная""}","{""between1And3"",""От 1 года до 3 лет""}","{""2"",""Санкт-Петербург"",""https://api.hh.ru/areas/2""}","{""Санкт-Петербург"",""проспект Обуховской Обороны"",""120с13"",59.864299,30.475346,null,""Санкт-Петербург, проспект Обуховской Обороны, 120с13"",{""Ломоносовская"",""Невско-Василеостровская"",""16.233"",""16"",59.877342,30.441715},[{""Ломоносовская"",""Невско-Василеостровская"",""16.233"",""16"",59.877342,30.441715}, {""Обухово"",""Невско-Василеостровская"",""16.235"",""16"",59.848709,30.457743}, … {""Рыбацкое"",""Невско-Василеостровская"",""16.236"",""16"",59.830986,30.501259}],""20021251""}","[{""ON_SITE"",""На месте работодателя""}]",[],"[{""115"",""Слесарь, сантехник""}]","{null,null,[]}",null,null,"{""Наличие квалификационного разряда не ниже 3 (слесарь механосборочных работ). Наличие подтверждающих документов о прохождении обучения /повышении квалификации. Чтение машиностроительных чертежей..."",""Слесарная обработка деталей, сборка, испытание сложных и опытных изделий, в т.ч. гидроприводов, редукторов, трубопроводов. Притирка и шабрение сопрягаемых поверхностей...""}","{""MAKEUP"",null}",null,null,null,2025-11-15 08:22:56,2025-11-15 08:22:56,0,0,2025-11-15 08:22:56,2025-11-15
"""100055445""","""Слесарь механосборочных работ …","{""shift"",""Сменный график""}",[],[],[],"[{""HOURS_12"",""12 часов""}]","[{""THREE_ON_THREE_OFF"",""3/3""}]",[],false,false,false,true,false,false,true,false,true,false,true,"""https://api.hh.ru/vacancies/10…","""https://hh.ru/vacancy/10005544…","""https://hh.ru/applicant/vacanc…",null,null,null,"{null,135000,""RUR"",true}","{null,135000,""RUR"",true,{""MONTH"",""За месяц""},{""TWICE_PER_MONTH"",""Два раза в месяц""}}","{""611846"",""«Научно-производственное объединение «Северо-Западный региональный центр Концерна ВКО «

## Describe

In [13]:
df_lazy_described = df_lazy.describe()

In [14]:
from IPython.display import display, HTML

display(HTML("""
<style>
/* classic notebook */
div.output_area pre {
  white-space: pre !important;   /* не переносить строки */
}

/* jupyterlab */
div.jp-OutputArea-output pre {
  white-space: pre !important;   /* не переносить строки */
}
</style>
"""))

with pl.Config(tbl_cols=-1, tbl_formatting='MARKDOWN', tbl_width_chars=-1):
    print(df_lazy_described)

shape: (9, 55)
| statistic  | id        | name                            | schedule | working_time_modes | working_time_intervals | working_days | working_hours | work_schedule_by_days | fly_in_fly_out_duration | is_adv_vacancy | internship | accept_temporary | accept_incomplete_resumes | premium  | has_test | show_contacts | response_letter_required | show_logo_in_search | archived | night_shifts | url                             | alternate_url                   | apply_alternate_url             | response_url                    | adv_response_url | immediate_redirect_url          | salary  | salary_range | employer | department | type     | employment | employment_form | experience | area     | address | work_format | relations | professional_roles | contacts | adv_context | sort_point_distance | snippet  | branding | insider_interview | video_vacancy | brand_snippet | date                | published_at_utc           | created_at_utc             | published_at_offset | created_at_o

## Check nulls

In [15]:
df_lazy.null_count().collect().glimpse()

Rows: 1
Columns: 54
$ id                        <u32> 0
$ name                      <u32> 0
$ schedule                  <u32> 0
$ working_time_modes        <u32> 0
$ working_time_intervals    <u32> 0
$ working_days              <u32> 0
$ working_hours             <u32> 0
$ work_schedule_by_days     <u32> 0
$ fly_in_fly_out_duration   <u32> 0
$ is_adv_vacancy            <u32> 0
$ internship                <u32> 0
$ accept_temporary          <u32> 0
$ accept_incomplete_resumes <u32> 0
$ premium                   <u32> 0
$ has_test                  <u32> 0
$ show_contacts             <u32> 0
$ response_letter_required  <u32> 0
$ show_logo_in_search       <u32> 69609
$ archived                  <u32> 0
$ night_shifts              <u32> 0
$ url                       <u32> 0
$ alternate_url             <u32> 0
$ apply_alternate_url       <u32> 0
$ response_url              <u32> 100478
$ adv_response_url          <u32> 123408
$ immediate_redirect_url    <u32> 100478
$ salary                 

## Слишком много null'ов в silver, проверка заполненности данных в bronze

In [19]:
bucket = s3.Bucket('bronze')

In [35]:
report_dt = "2025-12-07"
report_dt_objects = [o for o in bucket.objects.filter(Prefix=f"hh/vacancies/date={report_dt}")]
len(report_dt_objects)

1

In [36]:
for obj in report_dt_objects:
    s3path = f"s3://bronze/{obj.key}"
    lazy_bronze = pl.scan_ndjson(s3path, storage_options=storage_options, infer_schema_length=None)
    null_cnt = lazy_bronze.null_count().collect()
    print(null_cnt)
    

shape: (1, 49)
┌─────────────┬─────────────┬────────┬────────────┬───┬─────────────┬──────────┬─────┬─────────────┐
│ night_shift ┆ working_day ┆ salary ┆ created_at ┆ … ┆ fly_in_fly_ ┆ has_test ┆ url ┆ immediate_r │
│ s           ┆ s           ┆ ---    ┆ ---        ┆   ┆ out_duratio ┆ ---      ┆ --- ┆ edirect_url │
│ ---         ┆ ---         ┆ u32    ┆ u32        ┆   ┆ n           ┆ u32      ┆ u32 ┆ ---         │
│ u32         ┆ u32         ┆        ┆            ┆   ┆ ---         ┆          ┆     ┆ u32         │
│             ┆             ┆        ┆            ┆   ┆ u32         ┆          ┆     ┆             │
╞═════════════╪═════════════╪════════╪════════════╪═══╪═════════════╪══════════╪═════╪═════════════╡
│ 0           ┆ 0           ┆ 560    ┆ 0          ┆ … ┆ 0           ┆ 0        ┆ 0   ┆ 5121        │
└─────────────┴─────────────┴────────┴────────────┴───┴─────────────┴──────────┴─────┴─────────────┘


In [38]:
obj = report_dt_objects[0]
s3path = f"s3://bronze/{obj.key}"
lazy_bronze = pl.scan_ndjson(s3path, storage_options=storage_options, infer_schema_length=None)

In [40]:
silver_lazy = pl.scan_parquet(f"s3://silver/hh/vacancies/year=2025/month=12/day=7/part-13d2fa0f-b0db-4aca-85cb-c138f5639d7b.parquet", storage_options=storage_options)
silver_schema = set(silver_lazy.collect_schema().names())
bronze_schema = set(lazy_bronze.collect_schema().names())
len(silver_schema), len(bronze_schema)

(52, 49)

In [41]:
len(silver_schema & bronze_schema)

47

In [42]:
len(silver_schema - bronze_schema)

5

In [43]:
silver_schema - bronze_schema

{'created_at_offset',
 'created_at_utc',
 'date',
 'published_at_offset',
 'published_at_utc'}

In [48]:
'date' in lazy_bronze.collect_schema().names()

False

In [49]:
'date' in silver_lazy.collect_schema().names()

True

In [69]:
lazy_bronze = pl.scan_ndjson("s3://bronze/hh/vacancies/date=2025-12-07/*.jsonl.gz", storage_options=storage_options, infer_schema_length=None)
silver_lazy = pl.scan_parquet(f"s3://silver/hh/vacancies/year=2025/month=12/day=7/part-d696cca2-5e21-4dcb-b385-01df97955700.parquet", storage_options=storage_options)

In [70]:
'date' in lazy_bronze.collect_schema().names()

False

In [71]:
'date' in silver_lazy.collect_schema().names()

False

## professional_role count

In [84]:
df_lazy.head(1).collect().glimpse()

Rows: 1
Columns: 63
$ id                                    <str> '107445022'
$ name                                  <str> 'Кладовщик'
$ description                           <str> null
$ branded_description                   <str> null
$ key_skills                <list[struct[1]]> null
$ schedule                        <struct[2]> {'id': 'shift', 'name': 'Сменный график'}
$ accept_handicapped                   <bool> null
$ accept_kids                          <bool> null
$ experience                      <struct[2]> {'id': 'noExperience', 'name': 'Нет опыта'}
$ address                        <struct[10]> null
$ employer                        <struct[9]> {'id': '1942330', 'name': 'Пятёрочка', 'url': 'https://api.hh.ru/employers/1942330', 'alternate_url': 'https://hh.ru/employer/1942330', 'logo_urls': {'original': 'https://img.hhcdn.ru/employer-logo-original-round/7101933.png', '90': 'https://img.hhcdn.ru/employer-logo-round/7101934.png', '240': 'https://img.hhcdn.ru/employer-logo-ro

In [92]:
q = (df_lazy.with_columns(
        pl.col('professional_roles').list.get(0).struct.field('name').alias('professional_role')
    )
)
q.select('professional_role').unique().count().collect()

professional_role
u32
174
